In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as datetime
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import gc
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline

import os
print(os.listdir("../input"))

['sample_submission_v2.csv', 'test_v2.csv', 'train_v2.csv']


In [25]:
json_data=["device","geoNetwork","totals","trafficSource"]

gc.enable()

to_drop = ['socialEngagementType','device.browserVersion', 'device.browserSize', 'device.flashVersion', 'device.language', 
           'device.mobileDeviceBranding', 'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName', 'device.mobileDeviceModel',
           'device.mobileInputSelector', 'device.operatingSystemVersion', 'device.screenColors', 'device.screenResolution', 
           'geoNetwork.cityId', 'geoNetwork.latitude', 'geoNetwork.longitude','geoNetwork.networkLocation', 
           'trafficSource.adwordsClickInfo.criteriaParameters', 'trafficSource.adwordsClickInfo.gclId', 'trafficSource.campaign',
           'trafficSource.adwordsClickInfo.page', 'trafficSource.referralPath', 'trafficSource.adwordsClickInfo.slot',
           'trafficSource.adContent', 'trafficSource.keyword']

In [32]:
df_train = pd.read_csv("../input/train_v2.csv", sep=',',
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                    nrows = 1)
print(df_train.columns)

Index(['channelGrouping', 'customDimensions', 'date', 'device',
       'fullVisitorId', 'geoNetwork', 'hits', 'socialEngagementType', 'totals',
       'trafficSource', 'visitId', 'visitNumber', 'visitStartTime'],
      dtype='object')


In [34]:
def load_df(csv_path='train_v2.csv', features=features):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    ans = pd.DataFrame()
    dfs = pd.read_csv(csv_path, sep=',',
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                    chunksize = 100000)
    for df in dfs:
        df.reset_index(drop = True,inplace = True)
        for column in JSON_COLUMNS:
            column_as_df = json_normalize(df[column])
            column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
            df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
        use_df = df[features]
        del df
        gc.collect()
        ans = pd.concat([ans, use_df], axis = 0).reset_index(drop = True)
        print(ans.shape)
    return ans

In [35]:
df_train = load_df("../input/train_v2.csv", features=to_drop)
print(df_train.head())
list(df_train)

(100000, 25)
(200000, 25)
(300000, 25)
(400000, 25)
(500000, 25)
(600000, 25)
(700000, 25)
(800000, 25)
(900000, 25)
(1000000, 25)
(1100000, 25)
(1200000, 25)
(1300000, 25)
(1400000, 25)
(1500000, 25)
(1600000, 25)
(1700000, 25)
(1708337, 25)
   socialEngagementType          ...          trafficSource.keyword
0  Not Socially Engaged          ...                   water bottle
1  Not Socially Engaged          ...                            NaN
2  Not Socially Engaged          ...                            NaN
3  Not Socially Engaged          ...                 (not provided)
4  Not Socially Engaged          ...                 (not provided)

[5 rows x 25 columns]


['socialEngagementType',
 'device.browserVersion',
 'device.browserSize',
 'device.flashVersion',
 'device.language',
 'device.mobileDeviceBranding',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.operatingSystemVersion',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'trafficSource.adwordsClickInfo.criteriaParameters',
 'trafficSource.adwordsClickInfo.gclId',
 'trafficSource.campaign',
 'trafficSource.adwordsClickInfo.page',
 'trafficSource.referralPath',
 'trafficSource.adwordsClickInfo.slot',
 'trafficSource.adContent',
 'trafficSource.keyword']

In [36]:
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708337 entries, 0 to 1708336
Data columns (total 25 columns):
socialEngagementType                                 object
device.browserVersion                                object
device.browserSize                                   object
device.flashVersion                                  object
device.language                                      object
device.mobileDeviceBranding                          object
device.mobileDeviceInfo                              object
device.mobileDeviceMarketingName                     object
device.mobileDeviceModel                             object
device.mobileInputSelector                           object
device.operatingSystemVersion                        object
device.screenColors                                  object
device.screenResolution                              object
geoNetwork.cityId                                    object
geoNetwork.latitude                            

In [38]:
print(df_train['device.browserVersion'].isnull().sum(), df_train['device.browserVersion'].unique())

0 ['not available in demo dataset']


In [39]:
const_cols = [c for c in df_train.columns if df_train[c].nunique(dropna=True)==1 ]
const_cols

['socialEngagementType',
 'device.browserVersion',
 'device.browserSize',
 'device.flashVersion',
 'device.language',
 'device.mobileDeviceBranding',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.operatingSystemVersion',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'trafficSource.adwordsClickInfo.criteriaParameters']

In [42]:
for c in const_cols:
    print(c, '-----', df_train[c][0])

socialEngagementType ----- Not Socially Engaged
device.browserVersion ----- not available in demo dataset
device.browserSize ----- not available in demo dataset
device.flashVersion ----- not available in demo dataset
device.language ----- not available in demo dataset
device.mobileDeviceBranding ----- not available in demo dataset
device.mobileDeviceInfo ----- not available in demo dataset
device.mobileDeviceMarketingName ----- not available in demo dataset
device.mobileDeviceModel ----- not available in demo dataset
device.mobileInputSelector ----- not available in demo dataset
device.operatingSystemVersion ----- not available in demo dataset
device.screenColors ----- not available in demo dataset
device.screenResolution ----- not available in demo dataset
geoNetwork.cityId ----- not available in demo dataset
geoNetwork.latitude ----- not available in demo dataset
geoNetwork.longitude ----- not available in demo dataset
geoNetwork.networkLocation ----- not available in demo dataset
tr